In [1]:
from ROOT import TFile,TTree,TCanvas,TH1F, gStyle, TLatex, gPad, TLegend, TLorentzVector, TH2F, TLine, TF1, TBox, RDataFrame, TGraphErrors, TFitResultPtr, TF2
import ROOT
import numpy as np
from math import log10, floor

Welcome to JupyROOT 6.22/06


In [37]:
def round_sig(x, sig=2):
    if(x==0):
        return x
    return round(x, sig-int(floor(log10(abs(x))))-1)

def round_to_reference(x, y):
    if(y==0):
        return x
    return round(x, -int(floor(log10(y))))

def val_and_err(val,err,sig=2):
    err = round_sig(err,sig)
    val = round_to_reference(val,err)
    return val,err

In [2]:
def get_Mh_asym_plots(dataframe,cuts,Mhbins,Mggbins,Mggmin,Mggmax):
    nPlots = len(Mhbins-1)
    hists_total = []
    hists_plus = []
    hists_minus = []
    
    nCuts = len(cuts)
    total_cut = ""
    for i in range(nCuts):
        total_cut += cuts[i]
        total_cut += "&&"
    
    for i in range(nPlots-1):
        final_cut = total_cut + " Mdihadron > {:.5f} && Mdihadron < {:.5f}".format(Mhbins[i],Mhbins[i+1])
        hists_total.append(dataframe.Filter(final_cut).Histo1D(("ht{}".format(i),"",Mggbins,Mggmin,Mggmax),"Mdiphoton"))
        hists_plus.append(dataframe.Filter(final_cut+"&& helicity == 1").Histo1D(("hp{}".format(i),"",Mggbins,Mggmin,Mggmax),"Mdiphoton"))
        hists_minus.append(dataframe.Filter(final_cut+"&& helicity == -1").Histo1D(("hm{}".format(i),"",Mggbins,Mggmin,Mggmax),"Mdiphoton"))
    return hists_total, hists_plus, hists_minus

In [1]:
def do_pi0_basic_fit(h,fittype,nfitpars,Mggmin,Mggmax):
    # ========== Find maximum non-zero bin ============
    for i in range(h.GetValue().GetNbinsX()):
        if(h.GetValue().GetBinContent(i)==0):
            Mggmax = h.GetValue().GetBinCenter(i)
            break
    histint=h.Integral()
    h.Scale(1/histint)
    # ========== Create Fit Function ============
    f=TF1("f",fittype,Mggmin,Mggmax,nfitpars)
    f.SetNpx(1000)
    f.SetParameter(0,h.Integral())
    f.SetParameter(1,0.133)
    f.SetParameter(2,0.1)
    for i in range(3,nfitpars):
        f.SetParameter(i,10)
    f.SetParLimits(0,0,1)
    f.SetParLimits(1,0.129,0.135)
    f.SetParLimits(2,0.008,0.1)
    if(fittype=="crystalball(0)+pol2(5)" or fittype=="crystalball(0)+pol4(5)"  ):
        f.SetParLimits(3,-100,100)
        f.SetParLimits(4,-100,100)
    # ========== Perform the fit ============
    h.Fit(f,"Q0 SR")
    h.Fit(f,"Q0 SR")
    # ========== Get residuals plot ============
    bin1 = h.FindBin(Mggmin)
    bin2 = h.FindBin(Mggmax)
    residuals = TGraphErrors(bin2-bin1)
    for i in range(residuals.GetN()):
        BIN = bin1+i
        xval = h.GetBinCenter(BIN)
        yval = h.GetBinContent(BIN)
        yfit = f.Eval(h.GetBinCenter(BIN))
        yerr = h.GetBinError(BIN)
        if(yerr==0):
            continue
        residuals.SetPoint(i,xval,(yfit-yval)/yerr)
        residuals.SetPointError(i,0,0)
    return f,residuals.Clone()

In [1]:
def process_Mgg(hlist,fittype,nfitpars,Mggmin,Mggmax):
    fits = []
    ress = []
    for h in hlist:
        h.GetValue().SetLineColor(1)
        h.GetXaxis().SetNdivisions(508)
        h.GetXaxis().SetRangeUser(Mggmin,Mggmax)
        h.GetXaxis().SetLimits(Mggmin,Mggmax)
        h.GetXaxis().SetTitleSize(0)
        h.GetXaxis().SetLabelSize(0)
        h.SetMarkerStyle(20)
        h.SetTitle(";;Counts")
        f,res = do_pi0_basic_fit(h,fittype,nfitpars,Mggmin,Mggmax)
        fits.append(f)
        res.SetMarkerStyle(20)
        res.SetTitle(";M_{#gamma#gamma} [GeV];Residuals")
        res.GetXaxis().SetTitleSize(0.12)
        res.GetXaxis().SetLabelSize(0.12)
        res.GetYaxis().SetTitleSize(0.12)
        res.GetYaxis().SetLabelSize(0.12)
        res.GetYaxis().SetNdivisions(508)
        res.GetXaxis().SetTitleOffset(1)
        res.GetYaxis().SetTitleOffset(0.3)
        res.GetXaxis().SetLimits(Mggmin,Mggmax)
        ress.append(res)
    return fits, ress

In [1]:
def drawFitLatex(latex,x1,y1,ystep,fit,fittype,u):
    fitpars = []
    if(fittype=="crystalball(0)+pol2(5)"):
        fitpars = ["C","#mu","#sigma","#alpha","N","p0","p1","p2"]
    if(fittype=="crystalball(0)+pol4(5)"):
        fitpars = ["C","#mu","#sigma","#alpha","N","p0","p1","p2","p3","p4"]
    if(fittype=="gaus(0)+pol2(3)"):
        fitpars = ["C","#mu","#sigma","p0","p1","p2"]
    if(fittype=="gaus(0)+pol4(3)"):
        fitpars = ["C","#mu","#sigma","p0","p1","p2","p3","p4"]
        
    for i in range(len(fitpars)):
        val,err = val_and_err(fit.GetParameter(i),fit.GetParError(i),sig=2)
        latex.DrawLatexNDC(x1,y1-i*ystep,"{} = {} #pm {} ".format(fitpars[i],
                                                                  val,
                                                                  err
                                                                 ))
    
    latex.DrawLatexNDC(x1-ystep,y1-ystep*(len(fitpars)),"#chi^{2}/ndf = "+"{:.2f}/{:.0f} = {:.2f}".format(fit.GetChisquare(),
                                                                                               fit.GetNDF(),
                                                                                               fit.GetChisquare()/fit.GetNDF()))
    latex.DrawLatexNDC(x1-ystep,y1-ystep*(len(fitpars)+1),"Purity u = {:.3f}".format(u))
    return

In [12]:
def gethists_A_LU_Mh(dataframe,cuts,Mhbins,phihbins,phiRbins,signal_range,sideband_range):
    nPlots = len(Mhbins-1)
    hists_sig_plus = []
    hists_sig_minus = []
    
    hists_bg_plus = []
    hists_bg_minus = []
    
    nCuts = len(cuts)
    total_cut = ""
    for i in range(nCuts):
        total_cut += cuts[i]
        total_cut += "&&"
    
    for i in range(nPlots-1):
        final_cut = total_cut + " Mdihadron > {:.5f} && Mdihadron < {:.5f}".format(Mhbins[i],Mhbins[i+1])
        hists_sig_plus.append(d.Filter(final_cut + " && helicity == 1 && Mdiphoton > {} && Mdiphoton < {}".format(signal_range[0],signal_range[1]))
                          .Histo2D(("h1","h1",phihbins,-3.141592,3.141592,phiRbins,-3.141592,3.141592),"phi_h","phi_R"))
        
        hists_sig_minus.append(d.Filter(final_cut + " && helicity == -1 && Mdiphoton > {} && Mdiphoton < {}".format(signal_range[0],signal_range[1]))
                          .Histo2D(("h2","h2",phihbins,-3.141592,3.141592,phiRbins,-3.141592,3.141592),"phi_h","phi_R"))
        
        hists_bg_plus.append(d.Filter(final_cut + " && helicity == 1 && Mdiphoton > {} && Mdiphoton < {}".format(sideband_range[0],sideband_range[1]))
                          .Histo2D(("h3","h3",phihbins,-3.141592,3.141592,phiRbins,-3.141592,3.141592),"phi_h","phi_R"))
        
        hists_bg_minus.append(d.Filter(final_cut + " && helicity == -1 && Mdiphoton > {} && Mdiphoton < {}".format(sideband_range[0],sideband_range[1]))
                          .Histo2D(("h4","h4",phihbins,-3.141592,3.141592,phiRbins,-3.141592,3.141592),"phi_h","phi_R"))
        
    return hists_sig_plus, hists_sig_minus, hists_bg_plus, hists_bg_minus

In [14]:
def process_A_LU_Mh(uarray,hists_sig_plus,hists_sig_minus,hists_bg_plus,hists_bg_minus):
    
    f_sig=TF2("f_sig","[0]*sin(2*x-2*y)+[1]*sin(x-y)+[2]*sin(-x+2*y)+[3]*sin(y)+[4]*sin(x)+[5]*sin(2*x-y)+[6]*sin(3*x-2*y)",-3.141592,3.141592,-3.141592,3.141592)
    f_bg=TF2("f_bg","[0]*sin(2*x-2*y)+[1]*sin(x-y)+[2]*sin(-x+2*y)+[3]*sin(y)+[4]*sin(x)+[5]*sin(2*x-y)+[6]*sin(3*x-2*y)",-3.141592,3.141592,-3.141592,3.141592)
    
    h_sigs = []
    h_bgs = []
    f_sigs = []
    f_bgs = []
    
    fitparams = np.zeros((len(uarray),7))
    fiterrors = np.zeros((len(uarray),7))
    
    for i in range(len(uarray)):
        hsp1 = hists_sig_plus[i].GetValue().Clone()
        hsp2 = hists_sig_plus[i].GetValue().Clone()
        hsm = hists_sig_minus[i].GetValue().Clone()
        hbp1 = hists_bg_plus[i].GetValue().Clone()
        hbp2 = hists_bg_plus[i].GetValue().Clone()
        hbm = hists_bg_minus[i].GetValue().Clone()
        
        # Set Sumw2 for proper error propagation when dividing
        hsp1.Sumw2()
        hsp2.Sumw2()
        hsm.Sumw2()
        hbp1.Sumw2()
        hbp2.Sumw2()
        hbm.Sumw2()
        
        # For the sig+bg range (N+ - N-)/(N+ + N-)
        hsp1.Add(hsm,-1)
        hsp2.Add(hsm,+1)
        hsp1.Divide(hsp2)
        h_sigs.append(hsp1)
        hsp1.Fit(f_sig,"SR Q0")
        f_sigs.append(f_sig)
        fitparams_sig = np.array([f_sig.GetParameter(i) for i in range(7)])
        fiterrors_sig = np.array([f_sig.GetParError(i) for i in range(7)])
        
        # For the bg range (N+ - N-)/(N+ + N-)
        hbp1.Add(hbm,-1)
        hbp2.Add(hbm,+1)
        hbp1.Divide(hbp2)
        h_bgs.append(hbp1)
        hbp1.Fit(f_bg,"SR Q0")
        f_bgs.append(f_bg)
        fitparams_bg = np.array([f_bg.GetParameter(i) for i in range(7)])
        fiterrors_bg = np.array([f_bg.GetParError(i) for i in range(7)])
        
        # From equation 2 on page 10 of the pi+pi0 analysis note
        u=uarray[i]
        fitparams[i] = (1.0/u)*fitparams_sig - ((1.0-u)/(u))*fitparams_bg
        fiterrors[i] = np.sqrt( (1.0/u)**2 * fiterrors_sig**2 +  ((1.0-u)/(u)) **2 * fiterrors_bg**2   )
    return h_sigs, h_bgs, f_sigs, f_bgs, fitparams, fiterrors

In [4]:
mods = ["sin(2#phi_{h}-2#phi_{R})","sin(#phi_{h}-#phi_{R})", "sin(-#phi_{h}+2#phi_{R})" , "sin(#phi_{R})", "sin(#phi_{h})", "sin(2#phi_{h}-#phi{R})" , "sin(3#phi_{h}-2#phi_{R})"]

In [11]:
def get_u(method,h,fit,fittype,signal_range):
    # method : 0, 1, 2, 3: See Chris' pi0pi+ analysis note
    f_sig = 0
    f_bg = 0
    if(fittype=="crystalball(0)+pol2(5)"):
        f_sig = TF1("f_sig","crystalball",signal_range[0],signal_range[1])
        f_bg = TF1("f_bg","pol2",signal_range[0],signal_range[1])
        [f_sig.SetParameter(i,fit.GetParameter(i)) for i in range(5)]
        [f_bg.SetParameter(i,fit.GetParameter(i+5)) for i in range(3)]
    if(fittype=="crystalball(0)+pol4(5)"):
        f_sig = TF1("f_sig","crystalball",signal_range[0],signal_range[1])
        f_bg = TF1("f_bg","pol4",signal_range[0],signal_range[1])
        [f_sig.SetParameter(i,fit.GetParameter(i)) for i in range(5)]
        [f_bg.SetParameter(i,fit.GetParameter(i+5)) for i in range(5)]
    if(fittype=="gaus(0)+pol2(3)"):
        f_sig = TF1("f_sig","gaus",signal_range[0],signal_range[1])
        f_bg = TF1("f_bg","pol2",signal_range[0],signal_range[1])
        [f_sig.SetParameter(i,fit.GetParameter(i)) for i in range(3)]
        [f_bg.SetParameter(i,fit.GetParameter(i+3)) for i in range(3)]
    if(fittype=="gaus(0)+pol4(3)"):
        f_sig = TF1("f_sig","gaus",signal_range[0],signal_range[1])
        f_bg = TF1("f_bg","pol4",signal_range[0],signal_range[1])
        [f_sig.SetParameter(i,fit.GetParameter(i)) for i in range(3)]
        [f_bg.SetParameter(i,fit.GetParameter(i+3)) for i in range(5)]
    

    total_hist_sum = h.Integral(h.FindBin(signal_range[0]), h.FindBin(signal_range[1]))
    
    sig_fit_integral = f_sig.Integral(signal_range[0],signal_range[1])/h.GetBinWidth(1)
    bg_fit_integral = f_bg.Integral(signal_range[0],signal_range[1])/h.GetBinWidth(1)
    total_fit_integral = sig_fit_integral + bg_fit_integral
    
    u = 0
    if(method==0):
        u = sig_fit_integral/total_hist_sum
    elif(method==1):
        u = sig_fit_integral/total_fit_integral
    elif(method==2):
        u = 1 - bg_fit_integral/total_hist_sum
    elif(method==3):
        u = 1 - bg_fit_integral/total_fit_integral
    
    return u